In [1]:
configuration = {

    # experiment configuration
    'T': 150,
    'num_trajectory': 20, # 20

    # GMM configuration
    'gmm_max_samples': 20, # 20
    'gmm_max_clusters': 20,
    'gmm_min_samples_per_cluster': 40, # 40
    'gmm_prior_strength': 1.0,

    # eta, epsilon adjustment
    'init_eta': 1.0,
    'min_eta': 1e-8,
    'max_eta': 1e16,
    'eta_multiplier': 1e-4,

    'base_kl_step': 0.01,  # epsilon
    'init_kl_step_mult': 1.0,
    'min_kl_step_mult': 1e-1,
    'max_kl_step_mult': 1e2,

    # render
    'render_ok': False,

}

In [2]:
import logging

import numpy as np
import scipy.linalg


LOGGER = logging.getLogger(__name__)


def logsum(vec, axis=0, keepdims=True):
    maxv = np.max(vec, axis=axis, keepdims=keepdims)
    maxv[maxv == -float('inf')] = 0
    return np.log(np.sum(np.exp(vec-maxv), axis=axis, keepdims=keepdims)) + maxv


class GMM(object):
    """ Gaussian Mixture Model. """
    def __init__(self, init_sequential=False, eigreg=False, warmstart=True):
        self.init_sequential = init_sequential
        self.eigreg = eigreg
        self.warmstart = warmstart
        self.sigma = None

    def inference(self, pts):
        """
        Evaluate dynamics prior.
        Args:
            pts: A N x D array of points.
        """
        # Compute posterior cluster weights.
        logwts = self.clusterwts(pts)

        # Compute posterior mean and covariance.
        mu0, Phi = self.moments(logwts)

        # Set hyperparameters.
        m = self.N
        n0 = m - 2 - mu0.shape[0]

        # Normalize.
        m = float(m) / self.N
        n0 = float(n0) / self.N
        return mu0, Phi, m, n0

    def estep(self, data):
        """
        Compute log observation probabilities under GMM.
        Args:
            data: A N x D array of points.
        Returns:
            logobs: A N x K array of log probabilities (for each point
                on each cluster).
        """
        # Constants.
        N, D = data.shape
        K = self.sigma.shape[0]

        logobs = -0.5*np.ones((N, K))*D*np.log(2*np.pi)
        for i in range(K):
            mu, sigma = self.mu[i], self.sigma[i]
            L = scipy.linalg.cholesky(sigma, lower=True)
            logobs[:, i] -= np.sum(np.log(np.diag(L)))

            diff = (data - mu).T
            soln = scipy.linalg.solve_triangular(L, diff, lower=True)
            logobs[:, i] -= 0.5*np.sum(soln**2, axis=0)

        logobs += self.logmass.T
        return logobs

    def moments(self, logwts):
        """
        Compute the moments of the cluster mixture with logwts.
        Args:
            logwts: A K x 1 array of log cluster probabilities.
        Returns:
            mu: A (D,) mean vector.
            sigma: A D x D covariance matrix.
        """
        # Exponentiate.
        wts = np.exp(logwts)

        # Compute overall mean.
        mu = np.sum(self.mu * wts, axis=0)

        # Compute overall covariance.
        diff = self.mu - np.expand_dims(mu, axis=0)
        diff_expand = np.expand_dims(self.mu, axis=1) * \
                np.expand_dims(diff, axis=2)
        wts_expand = np.expand_dims(wts, axis=2)
        sigma = np.sum((self.sigma + diff_expand) * wts_expand, axis=0)
        return mu, sigma

    def clusterwts(self, data):
        """
        Compute cluster weights for specified points under GMM.
        Args:
            data: An N x D array of points
        Returns:
            A K x 1 array of average cluster log probabilities.
        """
        # Compute probability of each point under each cluster.
        logobs = self.estep(data)

        # Renormalize to get cluster weights.
        logwts = logobs - logsum(logobs, axis=1)

        # Average the cluster probabilities.
        logwts = logsum(logwts, axis=0) - np.log(data.shape[0])
        return logwts.T

    def update(self, data, K, max_iterations=100):
        """
        Run EM to update clusters.
        Args:
            data: An N x D data matrix, where N = number of data points.
            K: Number of clusters to use.
        """
        # Constants.
        N = data.shape[0]
        Do = data.shape[1]

        LOGGER.debug('Fitting GMM with %d clusters on %d points', K, N)

        if (not self.warmstart or self.sigma is None or
                K != self.sigma.shape[0]):
            # Initialization.
            LOGGER.debug('Initializing GMM.')
            self.sigma = np.zeros((K, Do, Do))
            self.mu = np.zeros((K, Do))
            self.logmass = np.log(1.0 / K) * np.ones((K, 1))
            self.mass = (1.0 / K) * np.ones((K, 1))
            self.N = data.shape[0]
            N = self.N

            # Set initial cluster indices.
            if not self.init_sequential:
                cidx = np.random.randint(0, K, size=(1, N))
            else:
                raise NotImplementedError()

            # Initialize.
            for i in range(K):
                cluster_idx = (cidx == i)[0]
                mu = np.mean(data[cluster_idx, :], axis=0)
                diff = (data[cluster_idx, :] - mu).T
                sigma = (1.0 / K) * (diff.dot(diff.T))
                self.mu[i, :] = mu
                self.sigma[i, :, :] = sigma + np.eye(Do) * 2e-6

        prevll = -float('inf')
        for itr in range(max_iterations):
            # E-step: compute cluster probabilities.
            logobs = self.estep(data)

            # Compute log-likelihood.
            ll = np.sum(logsum(logobs, axis=1))
            LOGGER.debug('GMM itr %d/%d. Log likelihood: %f',
                         itr, max_iterations, ll)
            if ll < prevll:
                # TODO: Why does log-likelihood decrease sometimes?
                LOGGER.debug('Log-likelihood decreased! Ending on itr=%d/%d',
                             itr, max_iterations)
                break
            if np.abs(ll-prevll) < 1e-5*prevll:
                LOGGER.debug('GMM converged on itr=%d/%d',
                             itr, max_iterations)
                break
            prevll = ll

            # Renormalize to get cluster weights.
            logw = logobs - logsum(logobs, axis=1)
            assert logw.shape == (N, K)

            # Renormalize again to get weights for refitting clusters.
            logwn = logw - logsum(logw, axis=0)
            assert logwn.shape == (N, K)
            w = np.exp(logwn)

            # M-step: update clusters.
            # Fit cluster mass.
            self.logmass = logsum(logw, axis=0).T
            self.logmass = self.logmass - logsum(self.logmass, axis=0)
            assert self.logmass.shape == (K, 1)
            self.mass = np.exp(self.logmass)
            # Reboot small clusters.
            w[:, (self.mass < (1.0 / K) * 1e-4)[:, 0]] = 1.0 / N
            # Fit cluster means.
            w_expand = np.expand_dims(w, axis=2)
            data_expand = np.expand_dims(data, axis=1)
            self.mu = np.sum(w_expand * data_expand, axis=0)
            # Fit covariances.
            wdata = data_expand * np.sqrt(w_expand)
            assert wdata.shape == (N, K, Do)
            for i in range(K):
                # Compute weighted outer product.
                XX = wdata[:, i, :].T.dot(wdata[:, i, :])
                mu = self.mu[i, :]
                self.sigma[i, :, :] = XX - np.outer(mu, mu)

                if self.eigreg:  # Use eigenvalue regularization.
                    raise NotImplementedError()
                else:  # Use quick and dirty regularization.
                    sigma = self.sigma[i, :, :]
                    self.sigma[i, :, :] = 0.5 * (sigma + sigma.T) + \
                            1e-6 * np.eye(Do)

In [3]:
class DynamicsPriorGMM(object):
    """
    A dynamics prior encoded as a GMM over [x_t, u_t, x_t+1] points.
    See:
        S. Levine*, C. Finn*, T. Darrell, P. Abbeel, "End-to-end
        training of Deep Visuomotor Policies", arXiv:1504.00702,
        Appendix A.3.
    """
    def __init__(self):
        """
        Hyperparameters:
            min_samples_per_cluster: Minimum samples per cluster.
            max_clusters: Maximum number of clusters to fit.
            max_samples: Maximum number of trajectories to use for
                fitting the GMM at any given time.
            strength: Adjusts the strength of the prior.
        """

        self.X = None
        self.U = None

        self.gmm = GMM()

        self._min_samp = configuration['gmm_min_samples_per_cluster']
        self._max_samples = configuration['gmm_max_samples']
        self._max_clusters = configuration['gmm_max_clusters']
        self._strength = configuration['gmm_prior_strength']


    def initial_state(self):
        """ Return dynamics prior for initial time step. """
        # Compute mean and covariance.
        mu0 = np.mean(self.X[:, 0, :], axis=0)
        Phi = np.diag(np.var(self.X[:, 0, :], axis=0))

        # Factor in multiplier.
        n0 = self.X.shape[2] * self._strength
        m = self.X.shape[2] * self._strength

        n0 = 1.0
        m = 1.0
        # Multiply Phi by m (since it was normalized before).
        Phi = Phi * m
        return mu0, Phi, m, n0

    def update(self, training_data):
        """
        Update prior with additional data.
        Args:
            X: A N x T x dX matrix of sequential state data.
            U: A N x T x dU matrix of sequential control data.
        """

        X = training_data.X
        U = training_data.U

        # Constants.
        T = X.shape[1] - 1

        # Append data to dataset.
        if self.X is None or self.U is None:
            self.X = X
            self.U = U
        else:
            self.X = np.concatenate([self.X, X], axis=0)
            self.U = np.concatenate([self.U, U], axis=0)

        # Remove excess samples from dataset.
        start = max(0, self.X.shape[0] - self._max_samples + 1)
        self.X = self.X[start:, :]
        self.U = self.U[start:, :]

        # Compute cluster dimensionality.
        Do = X.shape[2] + U.shape[2] + X.shape[2]  #TODO: Use Xtgt.

        # Create dataset.
        N = self.X.shape[0]
        xux = np.reshape(
            np.c_[self.X[:, :T, :], self.U[:, :T, :], self.X[:, 1:(T+1), :]],
            [T * N, Do]
        )

        # Choose number of clusters.
        K = int(max(2, min(self._max_clusters,
                           np.floor(float(N * T) / self._min_samp))))
        LOGGER.debug('Generating %d clusters for dynamics GMM.', K)

        # Update GMM.
        self.gmm.update(xux, K)

    def eval(self, Dx, Du, pts):
        """
        Evaluate prior.
        Args:
            pts: A N x Dx+Du+Dx matrix.
        """
        # Construct query data point by rearranging entries and adding
        # in reference.
        assert pts.shape[1] == Dx + Du + Dx

        # Perform query and fix mean.
        mu0, Phi, m, n0 = self.gmm.inference(pts)

        # Factor in multiplier.
        n0 = n0 * self._strength
        m = m * self._strength

        n0 = 1.0
        m = 1.0

        # Multiply Phi by m (since it was normalized before).
        Phi *= m
        return mu0, Phi, m, n0

In [4]:
import numpy as np
import scipy as sp

class ControlData(object):

    def __init__(self, Kt=None, kt=None, St=None, chol_St=None, inv_St=None, eta=None):

        self.Kt = Kt                # (T+1, action_dim, state_dim)
        self.kt = kt                # (T+1, action_dim)
        self.St = St                # (T+1, action_dim, action_dim)
        self.chol_St = chol_St      # (T+1, action_dim, action_dim)
        self.inv_St = inv_St        # (T+1, action_dim, action_dim)

        self.eta = eta


class LocalControl(object):

    def __init__(self, T, state_dim, action_dim):

        self.T = T  # t=0, 1, ..., T
        self.state_dim = state_dim
        self.action_dim = action_dim


    ## initialize local Gaussian controller
    def init(self):
        """"
            initialize the local Gaussian controller
        """
        Kt = np.zeros([self.T+1, self.action_dim, self.state_dim])
        kt = np.zeros([self.T+1, self.action_dim])
        St = np.zeros([self.T+1, self.action_dim, self.action_dim])
        chol_St = np.zeros([self.T+1, self.action_dim, self.action_dim])
        inv_St = np.zeros([self.T+1, self.action_dim, self.action_dim])

        T = self.T

        for t in range(T+1):
            St[t, :, :] = 1.0 * np.eye(self.action_dim)
            inv_St[t,:, :] = 1.0 / St[t, :, :]
            chol_St[t, :, :] = sp.linalg.cholesky(St[t, :, :])

        eta = configuration['init_eta']

        return ControlData(Kt, kt, St, chol_St, inv_St, eta)


    ## update local Gaussian LQR
    def update(self, control_data, dynamics_data, cost_param, goal_state, eta,
               kl_step_mult, MAX_iLQR_ITER=20):
        """
        @ description
            update each traj using ilqr.
            if the backward_pass fails, increase
            the kl penalty and recalculate the c_x, c_u, c_uu, c_ux, c_xx
        """

        T = self.T

        max_eta = configuration['max_eta']
        min_eta = configuration['min_eta']

        # set KL bound (epsilon)
        kl_bound = kl_step_mult * configuration['base_kl_step'] * (T+1)

        for itr in range(MAX_iLQR_ITER):

            # LQR backward pass
            backward_pass = self.backward(control_data, dynamics_data, eta, cost_param, goal_state)

            # LQR forward pass
            xu_mu, xu_cov = self.forward(backward_pass, dynamics_data)
            # compute KL divergence between local current and previous control laws

            kl_div = self.trajectory_kl(xu_mu, xu_cov, backward_pass, control_data)

            constraint = kl_div - kl_bound

            if abs(constraint) < 0.1 * kl_bound:
                print("KL converged iteration: ", itr)
                break

            # adjust eta
            if constraint < 0: # eta is too big
                max_eta = backward_pass.eta
                geo_mean = np.sqrt(min_eta*max_eta) # geometric mean
                new_eta = max(geo_mean, 0.1*max_eta)
            else: # eta is too small
                min_eta = backward_pass.eta
                geo_mean = np.sqrt(min_eta*max_eta)
                new_eta = min(10*min_eta, geo_mean)

            eta = new_eta

        return backward_pass

    ## LQR bqckward pass
    def backward(self, control_data, dynamics_data, eta, cost_param, goal_state):
        """
            LQR backward pass
            time-varying linear Gaussian controller
            ut = Kt * xt * kt + act_noise
            x_t+1 = fx * xt + fu ut + fc + noise
            if the backward_pass fails, increase
            the kl penalty and recalculate the cost_param
        """

        T = self.T
        state_dim = self.state_dim
        action_dim = self.action_dim

        # pull out dynamics
        fxu = dynamics_data.fxu            # (T+1, state_dim, (state_dim+action_dim))
        fc = dynamics_data.fc              # (T+1, state_dim)

        # initialization
        Kt = np.zeros((T+1, action_dim, state_dim))
        kt = np.zeros((T+1, action_dim))
        St = np.zeros((T+1, action_dim, action_dim)) # Quut_inv
        chol_St = np.zeros((T+1, action_dim, action_dim))
        Quut = np.zeros((T+1, action_dim, action_dim))

        slice_x = slice(state_dim)
        slice_u = slice(state_dim, state_dim + action_dim)
        eta0 = eta
        inc_eta = 1e-4  # in case of non positive-definite of Quu, incremental eta

        Quupd_err = True  # if Quu is non positive-definite matrix
        while Quupd_err:
            Quupd_err = False  # flip to true if Quu is non positive

            # initialization
            Vtt = np.zeros((T+1, state_dim, state_dim))
            vt = np.zeros((T+1, state_dim))
            Qtt = np.zeros((T+1, state_dim + action_dim, state_dim + action_dim))
            qt = np.zeros((T+1, state_dim + action_dim))

            # compute surrogate costs
            # Ctt: (T+1, state_dim+action_dim, state_dim+action_dim),  ct: (T+1, state_dim+action_dim)
            Ctt, ct = self.augment_cost(control_data, eta, cost_param, goal_state)

            for t in range(T, -1, -1):

                if t == T:
                    Qtt[t] = Ctt[t, :, :]
                    qt[t] = ct[t, :]
                else:
                    Qtt[t] = Ctt[t, :, :] + fxu[t, :, :].T.dot(Vtt[t+1, :, :]).dot(fxu[t, :, :])
                    qt[t] = ct[t, :] + fxu[t, :, :].T.dot(vt[t+1, :] + Vtt[t+1, :, :].dot(fc[t, :]))

                Qtt[t] = 0.5 * (Qtt[t] + Qtt[t].T)

                Quu = Qtt[t, slice_u, slice_u]
                Qux = Qtt[t, slice_u, slice_x]
                Qu = qt[t, slice_u]

                try:
                    # try to do Cholesky decomposittion for Quu
                    U = sp.linalg.cholesky(Quu)
                    L = U.T
                except:
                    # if decomposition fails, then Quu is not positive definite
                    Quupd_err = True
                    break

                Quut[t, :, :] = Quu
                # compute inverse of Quut
                Quu_inv = sp.linalg.solve_triangular(
                    U, sp.linalg.solve_triangular(L, np.eye(action_dim), lower=True)
                )
                St[t, :, :] = Quu_inv
                chol_St[t, :, :] = sp.linalg.cholesky(Quu_inv)

                #Kt[t] = -self.St[t].dot(Qtt[t, slice_u, slice_x])
                Kt[t, :, :] = -sp.linalg.solve_triangular(
                    U, sp.linalg.solve_triangular(L, Qux, lower=True)
                )
                #kt[t] = -self.St[t].dot(qt[t, slice_u])
                kt[t, :] = -sp.linalg.solve_triangular(
                    U, sp.linalg.solve_triangular(L, Qu, lower=True)
                )

                # compute value function
                Vtt[t, :, :] = Qtt[t, slice_x, slice_x] - Qux.T.dot(Quu_inv).dot(Qux)
                Vtt[t, :, :] = 0.5 * (Vtt[t, :, :] + Vtt[t, :, :].T)
                vt[t, :] = qt[t, slice_x] - Qux.T.dot(Quu_inv).dot(Qu)

                #Vtt[t, :, :] = Qtt[t, slice_x, slice_x] + Qtt[t, slice_x, slice_u].dot(Kt[t, :, :])
                #Vtt[t, :, :] = 0.5 * (Vtt[t, :, :] + Vtt[t, :, :].T)
                #vt[t, :] = qt[t, slice_x] + Qtt[t, slice_x, slice_u].dot(kt[t, :])

            # if Quut is not non positive-definite, increment eta
            if Quupd_err:
                eta = eta0 + inc_eta
                inc_eta *= 2.0
                print('Ooops ! Quu is not PD')

                if eta >= 1e16:
                    ValueError('Failed to find PD solution even for very large eta')

        return ControlData(Kt, kt, St, chol_St, Quut, eta)


    ## LQR forward pass
    def forward(self, backward_pass, dynamics_data):
        """
        @ description
            LQR forward pass
            time-varying linear Gaussian controller
            ut = Kt * xt * kt + act_noise
            x_t+1 = fx * xt + fu ut + fc + noise
            output : mu_t+1 = fx * mu_t + f_u * (K_t * mu_t + k_t) + fc
                     X_t+1 = fxu * [X_t X_t *K_t'; K_t*X_t K_t*X_t*K_t'+ Pt] * fxu' + E_t
        """

        T = self.T
        state_dim = self.state_dim
        action_dim = self.action_dim

        Kt, kt, St = backward_pass.Kt, backward_pass.kt, backward_pass.St

        # pull out dynamics
        fxu = dynamics_data.fxu            # (T+1, state_dim, (state_dim+action_dim))
        fc = dynamics_data.fc              # (T+1, state_dim)
        dyn_cov = dynamics_data.dyn_cov  # (T+1, state_dim, state_dim)

        # initialization
        xu_mu = np.zeros((T+1, state_dim + action_dim))
        xu_cov = np.zeros((T+1, state_dim + action_dim, state_dim + action_dim))

        slice_x = slice(state_dim)
        xu_mu[0, slice_x] = dynamics_data.x0mu
        xu_cov[0, slice_x, slice_x] = dynamics_data.x0cov

        for t in range(T+1):

            xu_mu[t,:] = np.hstack([
                xu_mu[t, slice_x],
                Kt[t,:,:].dot(xu_mu[t, slice_x]) + kt[t, :]
            ])

            xu_cov[t,:,:] = np.vstack([
                np.hstack([
                    xu_cov[t, slice_x, slice_x],
                    xu_cov[t, slice_x, slice_x].dot(Kt[t,:,:].T)
                ]),
                np.hstack([
                    Kt[t,:,:].dot(xu_cov[t, slice_x, slice_x]),
                    Kt[t,:,:].dot(xu_cov[t, slice_x, slice_x]).dot(Kt[t,:,:].T) + St[t,:,:]
                ])
            ])

            if t < T:
                xu_mu[t+1, slice_x] = fxu[t, :, :].dot(xu_mu[t, :]) + fc[t, :]
                xu_cov[t+1, slice_x, slice_x] = fxu[t,:,:].dot(xu_cov[t,:,:]).dot(fxu[t,:,:].T) + dyn_cov[t,:,:]

        return xu_mu, xu_cov


    ## compute costs for LQR backward pass for each local model
    def augment_cost(self, policy_data, eta, cost_param, goal_state):
        """
        @ description
            compute augmented cost used in the LQR backward pass.
         IN:
            policy_data = p_bar(ut|xt)
            original cost function = (x-goal_state).T * wx * (x-goal_state) + u.T * wu * u
            dual variable: eta
         OUT:
            modified Dtt (T+1, state_dim + action_dim, state_dim + action_dim)
            modified dt (T+1, state_dim + action_dim)
        """

        T = self.T
        state_dim = self.state_dim
        action_dim = self.action_dim

        slice_x = slice(state_dim)
        slice_u = slice(state_dim, state_dim + action_dim)

        # original cost
        Ctt = np.zeros((state_dim + action_dim, state_dim + action_dim))
        Ctt[slice_x, slice_x] = cost_param['wx'] * 2.0
        Ctt[slice_u, slice_u] = cost_param['wu'] * 2.0
        ct = np.zeros(state_dim + action_dim)
        ct[slice_x] = -2.0 * cost_param['wx'].dot(goal_state)

        # initialization
        Hessian = np.zeros((T+1, state_dim + action_dim, state_dim + action_dim))
        Jacobian = np.zeros((T+1, state_dim + action_dim))
        Dtt = np.zeros((T+1, state_dim + action_dim, state_dim + action_dim))
        dt = np.zeros((T+1, state_dim + action_dim))

        for t in range(T+1):
            inv_Sbar = policy_data.inv_St[t,:,:] # (action_dim, state_dim)
            KBar = policy_data.Kt[t, :, :]       # (action_dim, state_dim)
            kbar = policy_data.kt[t, :]          # (action_dim,)

            Hessian[t, :, :] = np.vstack([
                np.hstack([KBar.T.dot(inv_Sbar).dot(KBar), -KBar.T.dot(inv_Sbar)]),
                np.hstack([-inv_Sbar.dot(KBar), inv_Sbar])
            ])  # (state_dim+action_dim, state_dim+action_dim)

            Jacobian[t, :] = np.concatenate([
                KBar.T.dot(inv_Sbar).dot(kbar), -inv_Sbar.dot(kbar)
            ])   # (state_dim+action_dim,)

            Dtt[t,:,:] = Ctt / eta + Hessian[t, :, :]
            dt[t,:] = ct / eta + Jacobian[t, :]

        return Dtt, dt


    ## compute KL divergence between p and p_bar
    def trajectory_kl(self, xu_mu, xu_cov, backward_pass, policy_data):
        """
         KL divergence
        """

        T = self.T
        state_dim = self.state_dim
        action_dim = self.action_dim

        slice_x = slice(state_dim)

        # initialization of KL divergence for each time step
        kl_div_t = np.zeros(T+1)

        # for each time step
        for t in range(T+1):
            inv_Sbar = policy_data.inv_St[t, :, :]
            chol_Sbar = policy_data.chol_St[t, :, :]
            KBar= policy_data.Kt[t, :, :]
            kbar = policy_data.kt[t, :]

            Kt_new = backward_pass.Kt[t, :, :]
            kt_new = backward_pass.kt[t, :]
            St_new = backward_pass.St[t, :, :]
            chol_St_new = backward_pass.chol_St[t, :, :]

            K_diff = KBar - Kt_new
            k_diff = kbar - kt_new

            state_mu = xu_mu[t, slice_x]
            state_cov = xu_cov[t, slice_x, slice_x]

            logdet_Sbar = 2 * sum(np.log(np.diag(chol_Sbar)))
            logdet_St_new = 2 * sum(np.log(np.diag(chol_St_new)))

            kl_div_t[t] = max(
                0,
                0.5 * (
                    np.sum(np.diag(inv_Sbar.dot(St_new))) + \
                    logdet_Sbar - logdet_St_new - action_dim + \
                    k_diff.T.dot(inv_Sbar).dot(k_diff) + \
                    2 * k_diff.T.dot(inv_Sbar).dot(K_diff).dot(state_mu) + \
                    np.sum(np.diag(K_diff.T.dot(inv_Sbar).dot(K_diff).dot(state_cov))) + \
                    state_mu.T.dot(K_diff.T).dot(inv_Sbar).dot(K_diff).dot(state_mu)
                )
            )

        kl_div = np.sum(kl_div_t)

        return kl_div

In [5]:
class DynamicsData(object):

    def __init__(self, fxu=None, fc=None, dyn_cov=None, x0mu=None, x0cov=None):

        self.fxu = fxu              # (T+1, state_dim, state_dim + action_dim)
        self.fc = fc                # (T+1, state_dim)
        self.dyn_cov = dyn_cov      # (T+1, state_dim, state_dim)

        self.x0mu = x0mu
        self.x0cov = x0cov


class LocalDynamics(object):

    def __init__(self, T, state_dim, action_dim, prior):

        self.T = T
        self.state_dim = state_dim
        self.action_dim = action_dim

        self.prior = prior


    ## linear model update
    def update(self, training_data):

        X = training_data.X
        U = training_data.U
        N = X.shape[0]

        # update prior
        self.prior.update(training_data)

        # fit dynamics
        fxu, fc, dyn_cov = self.fit(X, U)

        # fit x0mu and x0cov
        x0 = X[:, 0, :]
        x0mu = np.mean(x0, axis=0)                 # (state_dim,)
        x0cov = np.diag(np.maximum(np.var(x0, axis=0), 1e-6))

        mu00, Phi0, priorm, n0 = self.prior.initial_state()
        x0cov += Phi0 + (N*priorm) / (N+priorm) * np.outer(x0mu-mu00, x0mu-mu00) / (N+n0)

        return DynamicsData(fxu, fc, dyn_cov, x0mu, x0cov)


    ## fit dynamics
    def fit(self, X, U, cov_reg=1e-6):

        N = X.shape[0]

        fxu = np.zeros([self.T+1, self.state_dim, self.state_dim + self.action_dim])
        fc = np.zeros([self.T+1, self.state_dim])
        dyn_cov = np.zeros([self.T+1, self.state_dim, self.state_dim])

        slice_xu = slice(self.state_dim + self.action_dim)
        slice_xux = slice(self.state_dim + self.action_dim, self.state_dim + self.action_dim + self.state_dim)

        # Fit dynamics with least squares regression.
        dwts = (1.0 / N) * np.ones(N)

        for t in range(self.T+1):

            # xux = [xt;  ut;  x_t+1]
            xux = np.c_[X[:, t, :], U[:, t, :], X[:, t+1, :]] # (N, state_dim+action_dim+state_dim)
            # compute Normal-inverse-Wishart prior
            mu0, Phi, mm, n0 = self.prior.eval(self.state_dim, self.action_dim, xux)

            # Build weights matrix.
            D = np.diag(dwts)

            # compute empirical mean and covariance (IMPORTANT !!)
            xux_mean = np.mean((xux.T * dwts).T, axis=0)   # <--       # ((state_dim+action_dim+state_dim),)
            #xux_mean = np.mean(xux, axis=0)
            diff = xux - xux_mean
            xux_cov = diff.T.dot(D).dot(diff)    # <-- # (state_dim+action_dim+state_dim, state_dim+action_dim+state_dim)
            #xux_cov = diff.T.dot(diff) / N
            xux_cov = 0.5 * (xux_cov + xux_cov.T)

            # MAP estimate of joint distribution
            map_cov = (Phi + N * xux_cov + (N * mm) / (N + mm) * np.outer(xux_mean-mu0, xux_mean-mu0)) / (N + n0)
            map_cov = 0.5 * (map_cov + map_cov.T)
            map_cov[slice_xu, slice_xu] += cov_reg * np.eye(self.state_dim+self.action_dim) # for matrix inverse

            map_mean = (mm * mu0 + n0 * xux_mean) / (mm + n0)
            #map_mean = xux_mean

            # compute model parameters
            fxut = np.linalg.solve(map_cov[slice_xu, slice_xu], map_cov[slice_xu, slice_xux]).T  # (state_dim, state_dim+action_dim)
            fct = map_mean[slice_xux] - fxut.dot(map_mean[slice_xu]) # (state_dim,)

            proc_cov = map_cov[slice_xux, slice_xux] - fxut.dot(map_cov[slice_xu, slice_xu]).dot(fxut.T)

            fxu[t, :, :] = fxut
            fc[t, :] = fct
            dyn_cov[t, :, :] = 0.5 * (proc_cov + proc_cov.T)

        return fxu, fc, dyn_cov

In [6]:
class TrainingData(object):

    """
        Definition of Training Data Structure: X, U, actual cost
    """
    def __init__(self, X=None, U=None, cost=None):
        self.X = X              # (N, T+2, state_dim)
        self.U = U              # (N, T+1, action_dim)
        self.cost = cost


## generate samples with specified local controller

class Sampler(object):

    """
        Generation of Training Data using current local control law
    """

    def __init__(self, env, N, T, state_dim, action_dim):

        self.env = env
        self.N = N
        self.T = T  # t=0, 1, ..., T
        self.state_dim = state_dim
        self.action_dim = action_dim

        # get action bound
        self.action_bound = env.action_space.high[0]

    ## generate samples with specified local Gaussian controller
    def generate(self, x0, local_controller, cost_param, goal_state):

        X = np.zeros((self.N, self.T+2, self.state_dim))   # X=(x0, x1, ..., xT, xT+1)
        U = np.zeros((self.N, self.T+1, self.action_dim))  # U = (u0, u1, ..., uT)

        Kt = local_controller.Kt
        kt = local_controller.kt
        St = local_controller.St

        state = np.zeros(self.state_dim)

        for traj_no in range(self.N):

            # reset the environment to the same initial conditions
            bad_init = True
            while bad_init:
                state = self.env.reset()  # shape of observation from gym (3,)
                x0err = state - x0
                if np.sqrt(x0err.T.dot(x0err)) < 0.1:  # x0=(state_dim,)
                    bad_init = False

            for t in range(self.T+1):
                # visualize the environment
                if configuration['render_ok']:
                    self.env.render()

                # compute action
                mean_action = Kt[t, :, :].dot(state) + kt[t, :]
                action = np.random.multivariate_normal(mean=mean_action, cov=St[t, :, :])
                action = np.clip(action, -self.action_bound, self.action_bound)

                # collect trajectory
                X[traj_no, t, :] = state
                U[traj_no, t, :] = action


                # observe next_state, shape of output of gym (state_dim,)
                state, _, _, _ = self.env.step(action)

                if t == self.T:
                    # collect trajectory
                    X[traj_no, t+1, :] = state

        cost = self.actual_cost(X, U, cost_param, goal_state)

        return TrainingData(X, U, cost)


    ## evaluate cost with real data
    def actual_cost(self, X, U, cost_param, goal_state):

        cost = 0
        for traj_no in range(self.N):
            for t in range(self.T+1):
                x = X[traj_no, t, :]
                u = U[traj_no, t, :]
                cost = cost + (x-goal_state).T.dot(cost_param['wx']).dot(x-goal_state) + \
                       u.T.dot(cost_param['wu']).dot(u)

        cost = cost / self.N

        return cost

In [7]:
import copy
import numpy as np
import math

class LQRFLMagent(object):

    def __init__(self, env):

        self.env = env
        # get state dimension
        self.state_dim = env.observation_space.shape[0]
        # get action dimension
        self.action_dim = env.action_space.shape[0]
        # get action bound
        self.action_bound = env.action_space.high[0]

        # initialize GMM
        self.prior = DynamicsPriorGMM()

        # goal state
        goal_ang = 0 * np.pi / 180.0
        self.goal_state = np.array([math.cos(goal_ang), math.sin(goal_ang), 0])

        # initial condition
        i_ang = -45.0 * np.pi / 180.0
        self.init_state = np.array([math.cos(i_ang), math.sin(i_ang), 0])

        self.N = configuration['num_trajectory']    # number of trajectories for each linear model
        self.T = configuration['T']                 # time horizon of trajectory for model fitting(0<=t<=self.T)

        # weightings in cost function
        self.cost_param = {
            'wx': np.diag([10.0, 0.01, 0.1]),  # (state_dim, state_dim)
            'wu': np.diag([0.001]),  # (action_dim, action_dim)
        }

        # epsilon
        self.kl_step_mult = configuration['init_kl_step_mult']

        # train data structure
        self.training_data = TrainingData()
        self.prev_training_data = TrainingData()
        self.sampler = Sampler(self.env, self.N, self.T, self.state_dim, self.action_dim)

        # create local dynamic models p(x_t+1|xt,ut)
        self.dynamics_data = DynamicsData()
        self.prev_dynamics_data = DynamicsData()
        self.local_dynamics = LocalDynamics(self.T, self.state_dim, self.action_dim, self.prior)

        # create local Gaussian controller p(ut|xt)
        self.control_data = ControlData()
        self.prev_control_data = ControlData()
        self.local_controller = LocalControl(self.T, self.state_dim, self.action_dim)

        # save the results
        self.save_costs = []

    ## update LQR
    def update(self, MAX_ITER):

        print("Now, regular iteration starts ...")

        for iter in range(int(MAX_ITER)):

            print("\niter = ", iter)

            # step 1: generate training data (by running previous LQR) -------------------
            #  1. initialize local control law
            #  2. generate training trajectory using prev_LQR

            x0 = self.init_state

            if iter == 0:
                self.control_data = self.local_controller.init()
                self.training_data = self.sampler.generate(x0, self.control_data, self.cost_param, self.goal_state)
            else:
                self.training_data = self.sampler.generate(x0, self.prev_control_data, self.cost_param, self.goal_state)

            # evaluate cost
            iter_cost = self.training_data.cost
            self.save_costs.append(iter_cost)
            print("     iter_cost  = ", iter_cost)

            # step 2: fit dynamics ------------------------------------------------------
            #  1. update GMM prior
            #  2. fit local linear time-varying dynamic model

            self.dynamics_data = self.local_dynamics.update(self.training_data)

            # step 3: update trajectory (update LQR) -----------------------------------
            #  1. design local LQR using local dynamic model

            if iter > 0:
                eta = self.prev_control_data.eta
                self.control_data = self.local_controller.update(self.prev_control_data,
                                                                self.dynamics_data, self.cost_param,
                                                                self.goal_state,
                                                                eta, self.kl_step_mult)

            # step 4: adjust kl_step -----------------------------------------------------

            if iter > 0:
                self._epsilon_adjust()

            # step 5: prepare next iteration ---------------------------------------------

            self._update_iteration_variables()


        #np.savetxt('./save_weights/pendulum_iter_cost.txt', self.save_costs)


    ## adjust KL step (epsilon)
    def _epsilon_adjust(self):

        _last_cost = self.prev_training_data.cost
        _cur_cost = self.training_data.cost

        _expected_cost = self.estimate_cost(self.control_data, self.dynamics_data)

        # compute predicted and actual improvement
        _expected_impr = _last_cost - _expected_cost
        _actual_impr = _last_cost - _cur_cost

        print("  cost last, expected, current = ", _last_cost, _expected_cost, _cur_cost)

        # adjust epsilon multiplier
        _mult = _expected_impr / (2.0 * max(1e-4, _expected_impr - _actual_impr))
        _mult = max(0.1, min(5.0, _mult))
        new_step = max(
                    min(_mult * self.kl_step_mult, configuration['max_kl_step_mult']),
                        configuration['min_kl_step_mult']
                    )

        self.kl_step_mult = new_step

        print(" epsilon_mult = ", new_step)


    ## previous <-- current
    def _update_iteration_variables(self):

        self.prev_training_data = copy.deepcopy(self.training_data)
        self.prev_dynamics_data = copy.deepcopy(self.dynamics_data)
        self.prev_control_data = copy.deepcopy(self.control_data)

        self.training_data = TrainingData()
        self.dynamics_data = DynamicsData()
        self.control_data = ControlData()


    ## estimate cost with local dynamics and local controller
    def estimate_cost(self, control_data, dynamics_data):

        T, state_dim, action_dim = self.T, self.state_dim, self.action_dim

        slice_x = slice(state_dim)
        slice_u = slice(state_dim, state_dim + action_dim)

        # original cost
        Ctt = np.zeros((state_dim + action_dim, state_dim + action_dim))
        Ctt[slice_x, slice_x] = self.cost_param['wx'] * 2.0
        Ctt[slice_u, slice_u] = self.cost_param['wu'] * 2.0
        ct = np.zeros((state_dim + action_dim))
        ct[slice_x] = -2.0 * self.cost_param['wx'].dot(self.goal_state)
        cc = self.goal_state.T.dot(self.cost_param['wx']).dot(self.goal_state)

        # pull out dynamics
        fxu = dynamics_data.fxu            # (T+1, state_dim, (state_dim+action_dim))
        fc = dynamics_data.fc              # (T+1, state_dim)
        dyn_cov = dynamics_data.dyn_cov  # (T+1, state_dim, state_dim)

        # pull out local controller
        Kt = control_data.Kt
        kt = control_data.kt
        St = control_data.St

        # initialization
        predicted_cost = np.zeros(T+1)
        xu_mu = np.zeros((T+1, state_dim + action_dim))
        xu_cov = np.zeros((T+1, state_dim + action_dim, state_dim + action_dim))
        xu_mu[0, slice_x] = dynamics_data.x0mu
        xu_cov[0, slice_x, slice_x] = dynamics_data.x0cov

        for t in range(T+1):

            xu_mu[t,:] = np.hstack([
                xu_mu[t, slice_x],
                Kt[t,:,:].dot(xu_mu[t, slice_x]) + kt[t, :]
            ])

            xu_cov[t,:,:] = np.vstack([
                np.hstack([
                    xu_cov[t, slice_x, slice_x], xu_cov[t, slice_x, slice_x].dot(Kt[t,:,:].T)
                ]),
                np.hstack([
                    Kt[t,:,:].dot(xu_cov[t, slice_x, slice_x]),
                    Kt[t,:,:].dot(xu_cov[t, slice_x, slice_x]).dot(Kt[t,:,:].T) + St[t,:,:]
                ])
            ])

            if t < T:
                xu_mu[t+1, slice_x] = fxu[t, :, :].dot(xu_mu[t, :]) + fc[t, :]
                xu_cov[t+1, slice_x, slice_x] = fxu[t,:,:].dot(xu_cov[t,:,:]).dot(fxu[t,:,:].T) + dyn_cov[t,:,:]

        for t in range(T+1):
            x = xu_mu[t, slice_x]
            u = xu_mu[t, slice_u]
            predicted_cost[t] = (x - self.goal_state).T.dot(self.cost_param['wx']).dot(x - self.goal_state) + \
                           u.T.dot(self.cost_param['wu']).dot(u) * np.sum(xu_cov[t, :, :]*Ctt)

        return predicted_cost.sum()

In [8]:
import gym

MAX_ITER = 60
env_name = 'Pendulum-v0'
env = gym.make(env_name)
agent = LQRFLMagent(env)

agent.update(MAX_ITER)

T = configuration['T']
Kt = agent.prev_control_data.Kt
kt = agent.prev_control_data.kt

print("\n\n Now play ................")
    # play
x0 = agent.init_state

play_iter = 5
save_gain = []

for pn in range(play_iter):

    print("     play number :", pn+1)

    if pn < 2:
        bad_init = True
        while bad_init:
            state = env.reset()  # shape of observation from gym (3,)
            x0err = state - x0
            if np.sqrt(x0err.T.dot(x0err)) < 0.1:  # x0=(state_dim,)
                bad_init = False
    else:
        state = env.reset()

        #state = env.reset()  # shape of observation from gym (3,)

    for time in range(T+1):
        env.render()
        action = Kt[time, :, :].dot(state) + kt[time, :]
        action = np.clip(action, -agent.action_bound, agent.action_bound)
        ang = math.atan2(state[1], state[0])

        print('Time: ', time, ', angle: ', ang*180.0/np.pi, 'action: ', action)

        save_gain.append([time, Kt[time, 0, 0], Kt[time, 0, 1], Kt[time, 0, 2], kt[time, 0]])

        state, reward, _, _ = env.step(action)

#np.savetxt('./save_weights/kalman_gain.txt', save_gain)


c:\users\박정은\gym\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Now, regular iteration starts ...

iter =  0
     iter_cost  =  2179.9982029739367

iter =  1
     iter_cost  =  2263.363652705283
KL converged iteration:  8
  cost last, expected, current =  2179.9982029739367 1133.252729712584 2263.363652705283
 epsilon_mult =  0.46311625344236895

iter =  2
     iter_cost  =  2325.8424567391758
KL converged iteration:  6
  cost last, expected, current =  2263.363652705283 1528.0371254596148 2325.8424567391758
 epsilon_mult =  0.21342403528980172

iter =  3
     iter_cost  =  2285.2783548485945
KL converged iteration:  4
  cost last, expected, current =  2325.8424567391758 1697.4173906278702 2285.2783548485945
 epsilon_mult =  0.11407545461410906

iter =  4
     iter_cost  =  2227.4295824258593
KL converged iteration:  6
  cost last, expected, current =  2285.2783548485945 1492.076778121832 2227.4295824258593
 epsilon_mult =  0.1

iter =  5
     iter_cost  =  2085.5585353224906
KL converged iteration:  5
  cost last, expected, current =  2227.4295824

     iter_cost  =  1566.4178037979027
KL converged iteration:  5
  cost last, expected, current =  1562.3189952844805 1340.9632552538585 1566.4178037979027
 epsilon_mult =  0.1

iter =  45
     iter_cost  =  1528.3543938297676
KL converged iteration:  4
  cost last, expected, current =  1566.4178037979027 1338.8894329368413 1528.3543938297676
 epsilon_mult =  0.1

iter =  46
     iter_cost  =  1417.8048298492092
KL converged iteration:  6
  cost last, expected, current =  1528.3543938297676 1330.0749940753378 1417.8048298492092
 epsilon_mult =  0.1130056827334694

iter =  47
     iter_cost  =  1397.0117510558302
KL converged iteration:  4
  cost last, expected, current =  1417.8048298492092 1314.009664671739 1397.0117510558302
 epsilon_mult =  0.1

iter =  48
     iter_cost  =  1362.6108837111237
KL converged iteration:  6
  cost last, expected, current =  1397.0117510558302 1316.9573614777105 1362.6108837111237
 epsilon_mult =  0.1

iter =  49
     iter_cost  =  1371.292909049191
KL c

Time:  91 , angle:  -18.763874181231508 action:  [-0.32009491]
Time:  92 , angle:  -23.07858550366501 action:  [-0.40225212]
Time:  93 , angle:  -28.408385512882663 action:  [-0.49058136]
Time:  94 , angle:  -34.971196177997065 action:  [-0.36532621]
Time:  95 , angle:  -42.92249085043695 action:  [-0.54037278]
Time:  96 , angle:  -52.56920257124856 action:  [-0.59081044]
Time:  97 , angle:  -64.17596735728746 action:  [-0.5541643]
Time:  98 , angle:  -77.95489176737829 action:  [-0.14373638]
Time:  99 , angle:  -93.89686983744474 action:  [-0.92271287]
Time:  100 , angle:  -112.37897873015856 action:  [-0.52451843]
Time:  101 , angle:  -133.07325504059747 action:  [-0.78317392]
Time:  102 , angle:  -155.67358129261916 action:  [-0.33034953]
Time:  103 , angle:  -179.30094398305786 action:  [-0.81392049]
Time:  104 , angle:  157.40477369466575 action:  [-0.25884732]
Time:  105 , angle:  135.20075892785823 action:  [-0.03760829]
Time:  106 , angle:  114.4945342338618 action:  [0.0258986

Time:  76 , angle:  3.169823456047225 action:  [0.27248821]
Time:  77 , angle:  2.016378489843098 action:  [0.2424766]
Time:  78 , angle:  1.042728753735836 action:  [0.16984141]
Time:  79 , angle:  0.1811631554162483 action:  [0.06533923]
Time:  80 , angle:  -0.6455313682173545 action:  [0.06892018]
Time:  81 , angle:  -1.4668165399855937 action:  [0.14708422]
Time:  82 , angle:  -2.2798965353919733 action:  [0.22760221]
Time:  83 , angle:  -3.0806452472726344 action:  [-0.2033243]
Time:  84 , angle:  -4.084234681736514 action:  [0.02954948]
Time:  85 , angle:  -5.228155289548715 action:  [-0.10994481]
Time:  86 , angle:  -6.615105066580855 action:  [0.07617534]
Time:  87 , angle:  -8.216836592804446 action:  [-0.01730415]
Time:  88 , angle:  -10.133080281716671 action:  [-0.03188672]
Time:  89 , angle:  -12.441039001407269 action:  [-0.15339986]
Time:  90 , angle:  -15.2777979490124 action:  [-0.19343243]
Time:  91 , angle:  -18.763830703529248 action:  [-0.32009587]
Time:  92 , angl

Time:  61 , angle:  131.44150682118973 action:  [-0.546498]
Time:  62 , angle:  110.60852422002435 action:  [-0.0371335]
Time:  63 , angle:  91.77068196042738 action:  [0.09746812]
Time:  64 , angle:  75.12228932606408 action:  [0.48286669]
Time:  65 , angle:  60.7579557877175 action:  [1.04334774]
Time:  66 , angle:  48.716751381937215 action:  [1.17224705]
Time:  67 , angle:  38.7938574759817 action:  [1.02875131]
Time:  68 , angle:  30.659173126753664 action:  [1.11322428]
Time:  69 , angle:  24.098493245585782 action:  [1.00199857]
Time:  70 , angle:  18.845674406219963 action:  [0.85178976]
Time:  71 , angle:  14.652923854336704 action:  [0.67123036]
Time:  72 , angle:  11.292127769093135 action:  [0.51154889]
Time:  73 , angle:  8.571872402499782 action:  [0.47198675]
Time:  74 , angle:  6.374685839621148 action:  [0.45835993]
Time:  75 , angle:  4.613022785193493 action:  [0.43435912]
Time:  76 , angle:  3.21081333415645 action:  [0.27546592]
Time:  77 , angle:  2.04731913417469

Time:  59 , angle:  -170.74726429595106 action:  [-2.]
Time:  60 , angle:  179.98973497780057 action:  [-2.]
Time:  61 , angle:  169.8676824971864 action:  [-2.]
Time:  62 , angle:  159.2641778872393 action:  [-2.]
Time:  63 , angle:  148.56196617210867 action:  [-2.]
Time:  64 , angle:  138.12097189733623 action:  [-2.]
Time:  65 , angle:  128.25485497968117 action:  [-2.]
Time:  66 , angle:  119.21651411443827 action:  [-2.]
Time:  67 , angle:  111.19398751489013 action:  [-2.]
Time:  68 , angle:  104.31528895705063 action:  [-2.]
Time:  69 , angle:  98.65903120742021 action:  [-2.]
Time:  70 , angle:  94.26743840069201 action:  [-2.]
Time:  71 , angle:  91.15904384169971 action:  [-2.]
Time:  72 , angle:  89.33936471580664 action:  [-2.]
Time:  73 , angle:  88.80869780621846 action:  [-2.]
Time:  74 , angle:  89.56672151999206 action:  [-2.]
Time:  75 , angle:  91.613838838525 action:  [-2.]
Time:  76 , angle:  94.94925893973924 action:  [-2.]
Time:  77 , angle:  99.56582303946641 a

Time:  54 , angle:  -84.53237790185756 action:  [-0.98452148]
Time:  55 , angle:  -100.22707225062865 action:  [-1.12923146]
Time:  56 , angle:  -118.52147260460437 action:  [-0.74727786]
Time:  57 , angle:  -139.02482679417392 action:  [-0.66247715]
Time:  58 , angle:  -161.22175980445923 action:  [-0.54325703]
Time:  59 , angle:  175.6562147730613 action:  [-0.22265102]
Time:  60 , angle:  152.8049616884706 action:  [-0.82173717]
Time:  61 , angle:  130.58254440291276 action:  [-0.48533032]
Time:  62 , angle:  109.78336173983429 action:  [0.02011195]
Time:  63 , angle:  91.01460145517106 action:  [0.15248581]
Time:  64 , angle:  74.4596219870387 action:  [0.51574096]
Time:  65 , angle:  60.196309134215404 action:  [1.05488428]
Time:  66 , angle:  48.2507042991556 action:  [1.17815878]
Time:  67 , angle:  38.414366201350774 action:  [1.02524703]
Time:  68 , angle:  30.353610705011253 action:  [1.10868917]
Time:  69 , angle:  23.85503846439643 action:  [0.99576361]
Time:  70 , angle:  